In [2]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point

In [3]:
# Load the road shapefiles
national_roads = gpd.read_file("C:/Users/HP/OneDrive - Université Mohammed VI Polytechnique/Desktop/UM6P/Internship/The Journey/Data/QGIS/data/national_routes/routes_nationales.shp")
regional_roads = gpd.read_file("C:/Users/HP/OneDrive - Université Mohammed VI Polytechnique/Desktop/UM6P/Internship/The Journey/Data/QGIS/data/routes_regionales/RoutesRegionales.shp")
provincial_roads = gpd.read_file("C:/Users/HP/OneDrive - Université Mohammed VI Polytechnique/Desktop/UM6P/Internship/The Journey/Data/QGIS/data/routes_provinciale/routes_provinciale.shp")

In [4]:
df = pd.read_csv('dams.csv')

In [5]:
# Convert to GeoDataFrame
dams_gdf = gpd.GeoDataFrame(
    df,
    geometry=gpd.points_from_xy(df.longitude, df.latitude),
    crs="EPSG:4326"  # assuming WGS84
)

In [6]:
metric_crs = "EPSG:32630"
dams_gdf = dams_gdf.to_crs(metric_crs)
national_roads = national_roads.to_crs(metric_crs)
regional_roads = regional_roads.to_crs(metric_crs)
provincial_roads = provincial_roads.to_crs(metric_crs)

In [7]:
# Function to calculate nearest distance
def get_nearest_distance(point, roads_gdf):
    distances = roads_gdf.geometry.distance(point)
    return distances.min()

In [8]:
# Calculate distances
dams_gdf['nearest_national_road'] = dams_gdf.geometry.apply(lambda p: get_nearest_distance(p, national_roads))
dams_gdf['nearest_regional_road'] = dams_gdf.geometry.apply(lambda p: get_nearest_distance(p, regional_roads))
dams_gdf['nearest_provincial_road'] = dams_gdf.geometry.apply(lambda p: get_nearest_distance(p, provincial_roads))

In [9]:
# Convert back to lat/lon if needed
dams_gdf = dams_gdf.to_crs("EPSG:4326")
dams_gdf['longitude'] = dams_gdf.geometry.x
dams_gdf['latitude'] = dams_gdf.geometry.y

In [10]:
# Save to CSV
dams_gdf[['longitude', 'latitude', 'nearest_national_road', 'nearest_regional_road', 'nearest_provincial_road']].to_csv("dam_nearest_roads.csv", index=False)